In [117]:
# Mengjie Ding & Shuai Wei @ Math 8650 Fall 2017 Clemson University
# Tic Tac Toe using Minimax algorithm
# Alpha-Beta pruning and Depth cutting off implemented


# Tic Tac Toe game class
class tic_tac_toe:
    # Game init: m*n board, who places k marks in a horizontal, vertical and diagonal win.
    # Board: represented by a python list
    # Symbol: X for player, O for computer; int number for empty place.
    def __init__(self, m = 3, n = 3, k = 3):
        self.m = m 
        self.n = n
        self.k = k
        self.turn = ''
        self.board = []
        for i in range(self.m * self.n):
            self.board.append(str(i+1))
    
    # Print the board
    def print_board(self):
        print()
        for i in range(self.m * self.n):
            if i == self.m * self.n - 1:
                print(self.board[i])
            elif (i+1)%self.n == 0:
                if self.board[i].isdigit():
                    if int(self.board[i])>9:
                        print(self.board[i])
                    else:
                        print(' '+self.board[i])
                else:
                    print(' '+self.board[i])
                print('- - '*self.n)
            else:
                if self.board[i].isdigit():
                    if int(self.board[i])>9:
                        print(self.board[i]+' | ',end='')
                    else:
                        print(' '+self.board[i]+' | ',end='')
                else:
                    print(' '+self.board[i]+' | ',end='')

        
    # Given a Game node, return the indices of all possible moves(i.e., the empty position)
    def get_possible_move(self):
        idx = []
        for i in range(len(self.board)):
            if self.board[i] != 'X' and self.board[i] != 'O':
                idx.append(i)
        return idx
            
    # Let the player choose the placing order
    def choose_Order(self,Computer,Player):
        if input('Wanna play first? (yes or no) ').lower().startswith('y'):
            self.turn = Player.Letter
        else:
            self.turn = Computer.Letter
    
    # Check if the game board is empty
    def _is_empty(self):
        
        for i in range(self.m * self.n):
            if self.board[i] == 'X' or self.board[i] == 'O':
                return False
        return True
      
    # Check if the game board is full
    def _is_full(self):
        
        for i in range(self.m * self.n):
            if self._is_free(i):
                return False
        return True
    
    # Check a specific place of the board is free or not
    def _is_free(self,i):
        # Return true if the element is free.
        return self.board[i] != 'X' and self.board[i] != 'O'
    
    #Check if 'char'(X or O) wins after the 'last_idx' move
    def _check_win(self,char,last_idx):
        
        # get the (i,j) coordinates of last_idx
        row = last_idx // self.n
        col = last_idx - row * self.n
        
        # Check if exist a horizontal k marks row
        k = 0
        for j in range(self.n):
            if self.board[row*self.m + j] != char:
                break
            k += 1
            if k >= self.k:
                return True
        # Check if exist a vertical k marks row
        k = 0
        for i in range(self.m):
            if self.board[i*self.m + col] != char:
                break
            k += 1
            if k >= self.k:
                return True
        
        # Check if exists a digonal k marks row
        k = 0
        i,j = row,col
        while i >= 0 and j >= 0:
            if self.board[i*self.m + j] != char:
                break
            k += 1
            i -= 1
            j -= 1
        i,j = row+1,col+1
        while i < self.m and j < self.n:
            if self.board[i*self.m + j] != char:
                break
            k += 1
            i += 1
            j += 1
        if k >= self.k:
            return True
        
        # Check if exists an anti-digonal marks row
        k = 0
        i,j = row,col
        while i < self.m and j >= 0:
            if self.board[i*self.m + j] != char:
                break
            k += 1
            i += 1
            j -= 1
        i,j = row-1,col+1
        while i >= 0 and j < self.n:
            if self.board[i*self.m + j] != char:
                break
            k += 1
            i -= 1
            j += 1
        if k >= self.k:
            return True
        
        return False
    
    # Start play the game between two players
    def play(self,Computer,Player):
        import timeit
        playing  = True
        while playing:
            
            if self.turn == Player.Letter: # Player's turn
                self.print_board()
                idx = Player.move(self)
                if self._check_win(Player.Letter,idx):
                    self.print_board()
                    print("You win!")
                    playing = False
                elif self._is_full():
                    self.print_board()
                    print('Tie!')
                    playing = False
                
                  
            else:               # Computer's turn
                # timeit for testing performance of AIcomputer
                start = timeit.default_timer()
                idx = Computer.move(self)
                stop = timeit.default_timer()
                #print(stop-start)
                if self._check_win(Computer.Letter,idx):
                    self.print_board()
                    print("You're defeated!")
                    playing = False
                elif self._is_full():
                    self.print_board()
                    print('Tie!')
                    playing = False

In [118]:
# Class for player
# Make move based on the input by player, then return that index
class player_type:
    def __init__(self,letter):
        self.Letter = letter
        
    def move(self,Game):
        enemy_letter = 'X' if self.Letter == 'O' else 'O'
        inputing = True
        while inputing:
            idx = input("Enter a number from 1 to " + str(Game.m * Game.n) + ": ")
            # Make sure 1.the input is a number; 2.in the range of board size; 3. not taken.
            if idx.isdigit() != True:
                print("Your input is not a number!")
                continue
            idx = int(idx) - 1
            if idx not in range(0,Game.m * Game.n):
                print("The inputing number is not between 1 and " + str(Game.m * Game.n) + ".")
            elif not Game._is_free(idx):
                print("The slot has been taken.")
            else:
                Game.board[idx] = self.Letter
                Game.turn = enemy_letter
                inputing = False
        
        return idx
    

In [119]:
# Minimax Function with no cutting off
# Given a Game node, calculate the value recursively
# Maxnode: maximum of the value of its children node
# Minnode: minimum of the value of its children node

def minimax(Node,last_idx,MAX,MIN):
    # Check if it's a leaf node   
    if Node._check_win(MIN,last_idx): # Player wins
        return -10 
    elif Node._check_win(MAX,last_idx): #Computer wins
        return 10 
    elif Node._is_full(): # Tie
        return 0  
    
    moves = Node.get_possible_move() # Get the children node
    
    if Node.turn == MAX: # MaxNode
        scores = []
        for idx in moves:
            
            Node.board[idx] = MAX
            Node.turn = MIN
            Val = minimax(Node,idx,MAX,MIN)
            Node.board[idx] = str(idx+1)
            Node.turn = MAX
            scores.append(Val)
        
        return max(scores)
    else:   # MinNode
        scores = []
        for idx in moves:
            
            Node.board[idx] = MIN
            Node.turn = MAX
            Val = minimax(Node,idx,MAX,MIN)
            Node.board[idx] = str(idx+1)
            Node.turn = MIN
            scores.append(Val)
            
        return min(scores)


In [120]:
# Minimax function with alpha-beta cutting off
def minimax_alpha_beta(Node,last_idx,MAX,MIN,alpha = -2000,beta = 2000):
    
    if Node._check_win(MIN,last_idx): # player win
        return -10 
    elif Node._check_win(MAX,last_idx): #computer win
        return 10 
    elif Node._is_full(): # Tie
        return 0 
    

    
    moves = Node.get_possible_move()
    
    if Node.turn == MAX: # This is a MaxNode
        bestVal = -2000
        
        for idx in moves:
            
            Node.board[idx] = MAX
            Node.turn = MIN
            Val = minimax_alpha_beta(Node,idx,MAX,MIN,alpha,beta)
            Node.board[idx] = str(idx+1)
            Node.turn = MAX
            bestVal = max(bestVal,Val)
            alpha = max(alpha,bestVal)
            if alpha >= beta:
                break
        
        return bestVal
    else:   # This is a MinNode
        bestVal = 2000
        for idx in moves:
            Node.board[idx] = MIN
            Node.turn = MAX
            Val = minimax_alpha_beta(Node,idx,MAX,MIN,alpha,beta)
            Node.board[idx] = str(idx+1)
            Node.turn = MIN
            bestVal = min(bestVal,Val)
            beta = min(beta,bestVal)
            if beta <= alpha:
                break
        
        return bestVal


In [121]:
# Minimax function with depth cutting off
# cut: the cut-off depth
def minimax_depth_cutting(Node,last_idx,MAX,MIN,depth=0,cut=1):
    
    if depth > cut:
        if Node.turn == MIN:
            return 1000
        elif Node.turn == MAX:
            return -1000
    
    if Node._check_win(MIN,last_idx): # player win
        return -10 
    elif Node._check_win(MAX,last_idx): #computer win
        return 10 
    elif Node._is_full(): # Tie
        return 0  
    

    
    moves = Node.get_possible_move()
    
    
    if Node.turn == MAX: # This is a MaxNode
        scores = []
        for idx in moves:
            
            Node.board[idx] = MAX
            Node.turn = MIN
            Val = minimax_depth_cutting(Node,idx,MAX,MIN,depth+1,cut)
            Node.board[idx] = str(idx+1)
            Node.turn = MAX
            scores.append(Val)
        
        return max(scores)
    else:   # This is a MinNode
        scores = []
        for idx in moves:
            
            Node.board[idx] = MIN
            Node.turn = MAX
            Val = minimax_depth_cutting(Node,idx,MAX,MIN,depth+1,cut)
            Node.board[idx] = str(idx+1)
            Node.turn = MIN
            scores.append(Val)
            
        return min(scores)


In [122]:
# Class for AIcomputer
# Given a Game node, make the move which has the biggest value and then return that index
# alg: different cutting off. 0: no ; 1: alpha-beta pruning; 2: depth cutting off
# cut: cut-off depth(if needed)
class AIcomputer:
    
    def __init__(self,letter,alg=0,cut=3):
        self.Letter = letter
        self.alg = alg
        self.cut = cut
    # alg: different cutting off. 0: no ; 1: alpha-beta pruning; 2: depth cutting off.
    def move(self,Game):
        import random
        from random import randint
       
        a = -20000
        enemy_letter = 'X' if self.Letter == 'O' else 'O'
        moves_idx = Game.get_possible_move()
        
        # If the computer play first, then make a random move
        if len(moves_idx) == Game.m * Game.n:
            random.seed()
            idx = randint(0, Game.m * Game.n - 1)
            Game.board[idx] = self.Letter
            Game.turn = enemy_letter
            return idx
        
        
        for i in moves_idx:
            
            Game.board[i] = self.Letter
            Game.turn = enemy_letter
            if self.alg == 0: val = minimax(Game,i,self.Letter,enemy_letter)
            if self.alg == 1: val = minimax_alpha_beta(Game,i,self.Letter,enemy_letter)
            if self.alg == 2: val = val = minimax_depth_cutting(Game,i,self.Letter,enemy_letter,0,self.cut)
            Game.board[i] = str(i+1)
            Game.turn = self.Letter
            
            if val > a:
                a = val
                choices = []
                choices.append(i)
            elif val == a:
                choices.append(i)
            
        # Randomly choose one from collections of moves which have biggest value
        idx = random.choice(choices)
        
        Game.board[idx] = self.Letter
        Game.turn = enemy_letter
        
        return idx
        

In [129]:
# Construct the game tic_tac_toe(m,n,k): m*n board with k in a row
# AI algorithm selection AIcomputer('O',a,b)
# a:0 for minimax; 1 for minimax with alpha-beta pruning; 2 for depth cutting off with b the cut-off depth.
# for 3-by-3 case, use minimax or minimax with alpha-beta pruning.
# for 4-by-4 or high-order, use minimax with depth cutting off.
try:
    while True:
        Game = tic_tac_toe(3,3,3)
        Computer = AIcomputer('O',1,3)
        Player = player_type('X')
        #Player = AIcomputer('X',1)
        Game.choose_Order(Computer,Player)
        Game.play(Computer,Player)
        if not input('Wanna to play again? (yes or no) ').lower().startswith('y'):
            break
except KeyboardInterrupt:
    print('interrupted!')
    

Wanna play first? (yes or no) y

 1 |  2 |  3
- - - - - - 
 4 |  5 |  6
- - - - - - 
 7 |  8 | 9
Enter a number from 1 to 9: 5

 1 |  2 |  3
- - - - - - 
 4 |  X |  6
- - - - - - 
 7 |  8 | O
Enter a number from 1 to 9: 3

 1 |  2 |  X
- - - - - - 
 4 |  X |  6
- - - - - - 
 O |  8 | O
Enter a number from 1 to 9: 2

 1 |  X |  X
- - - - - - 
 4 |  X |  6
- - - - - - 
 O |  O | O
You're defeated!
Wanna to play again? (yes or no) no
